In [ ]:
!pip install moviepy pytube ffmpeg-python asyncffmpeg asynccpu --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 910.3 kB/s eta 0:00:00


In [ ]:
!pip install SpeechRecognition --quiet
!pip install pydub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 39.0 MB/s eta 0:00:00


In [ ]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from pydub.utils import make_chunks

In [ ]:
from pytube import YouTube
import ffmpeg
import requests
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
import json
import os
import warnings
warnings.filterwarnings('ignore')
from moviepy.editor import *
import concurrent.futures
from multiprocessing import process
from threading import Thread
from asynccpu import ProcessTaskPoolExecutor
from asyncffmpeg import FFmpegCoroutineFactory, StreamSpec
import asyncio
import tarfile
from zipfile import ZipFile

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Check if mp4 file is playable
def is_mp4_playable(file_path):
  try:
      # Probe the file to get information about it
      probe = ffmpeg.probe(file_path)
      # Check if the file format is recognized as video
      if 'streams' in probe and any(stream['codec_type'] == 'video' for stream in probe['streams']):
          print("MP4 file is playable.")
          return True
      else:
          print("MP4 file is corrupt or non-playable.")
          return False
  except ffmpeg.Error as e:
      print("Error occurred:", e.stderr)
      return False

In [ ]:
# Extract audio from given video
def extract_audio(video_url, output_folder):
  if output_folder:
    audio, err = (
      ffmpeg
      .input(video_url)
      .output("pipe:", format='mp3', acodec='libmp3lame', audio_bitrate='320k')
      .run(capture_stdout=True)
    )
    with open(output_folder, 'wb') as f:
        f.write(audio)
    print("Audio extraction complete")
  else:
    print("Audio extraction not done - no output file given")

In [ ]:
# Extract frames from given video
def extract_frames(video_url, output_folder, fps=0.1):
  if output_folder:
    (
        ffmpeg
        .input(video_url)
        .filter('fps', fps=fps)
        .output(output_folder + '/frame%d.png', **{'qscale:v': 2})
        .run()
    )
    print("video frames extraction complete")
  else:
    print("Video frames extraction not done - no output file given")

In [ ]:
# Process given video/webpage link to extract correct video path and make sure the video is accessible
def process_video(link):

  if len(urlparse(link).netloc) > 0:
    if urlparse(link).netloc == "www.youtube.com":
      yt = YouTube(link)
      video_path = yt.streams[0].url
      return video_path

    elif urlparse(link).netloc == "www.linkedin.com":
      r = requests.get(link)
      soup = BeautifulSoup(r.content, 'html.parser')
      data_linkedin = json.loads(soup.find('script', type='application/ld+json').text)
      if data_linkedin['isAccessibleForFree'] == True:
        video_path = json.loads(soup.video['data-sources'])[0]['src']
        return video_path
      else:
        print("Sorry! Can't process video. Please make sure the video is free for access.")
        return

    elif urlparse(link).netloc == "www.coursera.org":
      r = requests.get(link)
      soup = BeautifulSoup(r.content, 'html.parser')
      data_coursera = json.loads(soup.find('script', type='application/ld+json').text)
      video_path = data_coursera['@graph'][1]['contentURL']
      return video_path

    else:
      print("Sorry! Can't process video. Please make sure the video link is valid.")
      return

  else:
    if(is_mp4_playable(link)):
      video_path = link
      return video_path
    else:
      print("Sorry! Can't process video. Please make sure the video file exists and is not corrupted.")
      return

In [ ]:
# Parallel process audio and frames extraction
async def main(video_link, output_file_path_audio,output_file_path_frames):

  video_path = process_video(video_link)
  if video_path:
      awaitables = [
            asyncio.to_thread(extract_audio, video_path, output_file_path_audio),
            #asyncio.to_thread(extract_frames, video_path, output_file_path_frames)
      ]
      await asyncio.gather(*awaitables)

In [ ]:
# Testing out with an example video
if __name__ == '__main__':
  video_link=input("Video link: ")
  output_file_path_audio=input("MP3 file path to store audio: ")
  output_file_path_frames=input("Folder path to store video frames: ")
  await main(video_link, output_file_path_audio,output_file_path_frames)

https://www.youtube.com/watch?v=Pu5CNKweuHw
/content/drive/My Drive/LLM_Project/youtube_audio_2.mp3
/content/drive/My Drive/LLM_Project/frames
video frames extraction complete
Audio extraction complete


### TVSum Dataset:

In [ ]:
# Extracting TVSum data
os.chdir('/content/drive/My Drive/')

with tarfile.open("tvsum50_ver_1_1.tgz", 'r:gz') as tar:
    tar.extractall( path= "/content/drive/My Drive/LLM_Project/tvsum" )

In [ ]:
# Extracting videos from TVSum data
os.chdir('/content/drive/MyDrive/LLM_Project/tvsum/ydata-tvsum50-v1_1/')

with ZipFile("ydata-tvsum50-video.zip", 'r') as zObject:
    zObject.extractall( path= "ydata-tvsum50-video" )

In [ ]:
# Getting list of videos
video_path = '/content/drive/MyDrive/LLM_Project/tvsum/ydata-tvsum50-v1_1/ydata-tvsum50-video/video/'
videos=os.listdir(video_path)

In [ ]:
# Extracting audio and frames from each video and storing in individual folders (This took around 13 min)
if __name__ == '__main__':
  for video in videos:
    print("\n",video,":")
    output_file_path_frames = '/content/drive/My Drive/LLM_Project/tvsum/ydata-tvsum50-v1_1/ydata-tvsum50-video/frames/'+video.split('.')[0]
    output_file_path_audio = '/content/drive/My Drive/LLM_Project/tvsum/ydata-tvsum50-v1_1/ydata-tvsum50-video/audio'
    if not os.path.exists(output_file_path_frames):
      os.makedirs(output_file_path_frames)
    if not os.path.exists(output_file_path_audio):
      os.makedirs(output_file_path_audio)
    await main(video_path+video, output_file_path_audio+'/'+video.split('.')[0]+'.mp3', output_file_path_frames)

### SumMe Dataset

In [ ]:
# Extracting SumMe data
os.chdir('/content/drive/MyDrive/')

with ZipFile("SumMe.zip", 'r') as zObject:
    zObject.extractall( path= "/content/drive/MyDrive/LLM_Project/SumMe" )

In [ ]:
# Getting list of videos
video_path = '/content/drive/MyDrive/LLM_Project/SumMe/SumMe/videos/'
videos=os.listdir(video_path)

In [ ]:
# Extracting audio and frames from each video and storing in individual folders (This took around 8 min. Two of the videos were producing error when extracting. Will look into it)
if __name__ == '__main__':
  for video in videos:
    if video.split('.')[1] == "mp4":
      print("\n",video,":")
      output_file_path_frames = '/content/drive/My Drive/LLM_Project/SumMe/SumMe/frames/'+video.split('.')[0]
      output_file_path_audio = '/content/drive/My Drive/LLM_Project/SumMe/SumMe/audio'
      if not os.path.exists(output_file_path_frames):
        os.makedirs(output_file_path_frames)
      if not os.path.exists(output_file_path_audio):
        os.makedirs(output_file_path_audio)
      await main(video_path+video, output_file_path_audio+'/'+video.split('.')[0]+'.mp3', output_file_path_frames)

## Audio to text conversion and summarization

In [ ]:
def divide_audio_chunks(audio_file, chunksize = 30000):
    mp3_audio = AudioSegment.from_mp3(audio_file)

    # Split the audio into chunks
    chunks = make_chunks(mp3_audio, chunksize)

    return chunks

In [ ]:
def process_chunks(audio_chunks):
  whole_speech = ""
  recognizer = sr.Recognizer()
  for i, chunk in enumerate(audio_chunks):
    audio = chunk.export(format="wav")
    with sr.AudioFile(audio) as source:
      audio_data = recognizer.record(source)
      try:
        text = recognizer.recognize_google(audio_data)
        print(f"Chunk {i+1}: {text}")
        whole_speech += text
      except sr.UnknownValueError:
        print(f"Chunk {i+1}, Google Speech Recognition could not understand audio")
      except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
  return whole_speech

In [ ]:
yt_videos = ["https://www.youtube.com/watch?v=U1omz0B9FTw", "https://www.youtube.com/watch?v=d_qvLDhkg00",
"https://www.youtube.com/watch?v=h5id4erwD4s",
"https://www.youtube.com/watch?v=tl30y5OOfqQ",
"https://www.youtube.com/watch?v=6bJTEZnTT5A",
"https://www.youtube.com/watch?v=SZorAJ4I-sA",
"https://www.youtube.com/watch?v=V_2QqOEwzYU",
"https://www.youtube.com/watch?v=yw-E__nDkKU",
"https://www.youtube.com/watch?v=en2bmeB4QUo",
"https://www.youtube.com/watch?v=ch5EQgzfroo"]

In [ ]:
# Extracting audio and frames from each video and storing in individual folders (This took around 8 min. Two of the videos were producing error when extracting. Will look into it)
if __name__ == '__main__':
  for i in range(len(yt_videos)):
      output_file_path_frames = f"/content/drive/My Drive/LLM_Project/YT/frames/{i}"
      output_file_path_audio = '/content/drive/My Drive/LLM_Project/YT/audio'
      if not os.path.exists(output_file_path_frames):
        os.makedirs(output_file_path_frames)
      if not os.path.exists(output_file_path_audio):
        os.makedirs(output_file_path_audio)
      await main(yt_videos[i], output_file_path_audio+'/'+str(i)+'.mp3', output_file_path_frames)

video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete
video frames extraction complete
Audio extraction complete


In [ ]:
transcripts = []
audio_path = '/content/drive/My Drive/LLM_Project/YT/audio/'
audios=os.listdir(audio_path)

In [ ]:
for audio in audios:
  audio_chunks = divide_audio_chunks(audio_path+audio)
  transcripts.append(process_chunks(audio_chunks))

Chunk 21: you're supported and even though this is technically a massive IP it is an original thing that Cameron just kind of invented and it's this world that's supported by two films and over a decade of work so you know you can spend a few hours and go see it guys thank you so much as always for watching look forward to more videos very soon and if you like this you can click right here and get stuck by nice
Chunk 22, Google Speech Recognition could not understand audio


In [ ]:
from transformers import pipeline

bart_summaries=[]
# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_long_text(text, max_length=1024, min_length=50):
    """
    Summarize a long text by splitting it into chunks and generating summaries for each chunk.

    Args:
        text (str): The input text to be summarized.
        max_length (int): The maximum length of the input text for the summarization model.
        min_length (int): The minimum length of the generated summaries.

    Returns:
        str: The combined summary of the input text.
    """
    # Split the text into chunks
    chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]

    # Generate summaries for each chunk
    summaries = []

    for chunk in chunks:
        #chunk = f"Write a summary of the following text delimited by triple backticks.\nReturn your response which covers the key points of the text\n```{chunk}```\nSUMMARY:"
        summary = summarizer(chunk, max_length=min_length, min_length=min_length, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    # Combine the summaries
    combined_summary = ' '.join(summaries)

    return combined_summary

# Example usage

for transcript in transcripts:
  bart_summaries.append(summarize_long_text(transcript))

Your max_length is set to 50, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 50, but your input_length is only 11. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is set to 50, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 50, but your input_length is only 5. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


In [ ]:
transcripts[0]

"what's going on everyone in this video we will learn what is logistic regression where it is used and with the simple example we will try to understand how to apply logistic regression so let's get started I am Jay Patel and I upload new videos on machine learning every week so make sure to hit the red subscribe button and also hit the Bell icon and for now let's jump to this board logistic regression is a machine learning model which is used for the binary classificationwhat is binary classification binary classification is where our output predictions can only take one of the two possible values for example it can be either 0 or 1 or it can be either true or false was some of the applications where the logistic regression can be used in real world can be to classify the email is a spam or not where to find out if the transaction is fraud or notor it can be also to find out if the person having a disease or not or to find out if the tumor is a malignant or not so you can see that log

In [ ]:
bart_summaries[0]

"logistic regression is a machine learning model which is used for the binary classification. binary classification is where our output predictions can only take one of the two possible values. logistic regression can be used in real world to classify the email is Simplicity is an open-source, free-to-use computer program. It can be used to predict if a person is having a heart disease or not based on their age. The program can also predict if the person is Let's see if you want to predict if the person having this age has a heart disease or not then this method will give us actually the wrong way because it is classifying zero sogo. To overcome this one thing we can do The formula for Discordis 1/11 + - x is y equal to mx + C.  0.5 as the value when our ex is 0 and whenever X is very large it takes roughly one as thevalue not exactly  Matrix as y equals to W transpose x + B now this W represent the Matrix of all these W's and it is of the shape and comma one in other words we can pass

In [ ]:
transcript_summary_dict={}
for i in range(len(bart_summaries)):
  transcript_summary_dict[transcripts[i]] = bart_summaries[i]

In [ ]:
transcript_summary_dict

In [ ]:
import csv

input_text=[]
summaries=[]
for key,value in transcript_summary_dict.items():
    input_text.append(key)
    summaries.append(value)
# Path where you want to save the CSV file which contains text and summaries
file_path = '/content/drive/My Drive/LLM_Project/YT/output.csv'
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['input_text', 'summaries'])
    # Write data rows
    for a, b in zip(input_text, summaries):
        writer.writerow([a, b])

In [ ]:
import re

def clean_whitespace(text):
    """Replace more than two consecutive whitespaces with a single space."""
    return re.sub(r'\s{2,}', ' ', text)

def clean_csv(input_file_path, output_file_path):
    """Read CSV, clean data, and write to a new CSV file."""
    with open(input_file_path, mode='r', newline='') as infile:
      with open(output_file_path, mode='w', newline='') as outfile:
          reader = csv.reader(infile)
          writer = csv.writer(outfile)
          headers = next(reader)
          writer.writerow(headers)
          for row in reader:
              cleaned_row = [clean_whitespace(cell) for cell in row]
              writer.writerow(cleaned_row)


input_csv_path = '/content/drive/My Drive/LLM_Project/YT/output.csv'
output_csv_path = '/content/drive/My Drive/LLM_Project/YT/cleaned_bart_summaries.csv'

# Call the function to clean the CSV
clean_csv(input_csv_path, output_csv_path)

In [ ]:
# from openai import OpenAI
# client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "key_here"))

In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/My Drive/LLM_Project/YT/cleaned_bart_summaries.csv')



# Evaluation prompt template based on G-Eval
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully.
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""

# Function to simulate evaluation score fetching
def get_geval_score(
    criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content

# Evaluation criteria and steps as per different metrics
evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS)
}

# Data structure to hold the evaluation results
#data = {"Evaluation Type": [], "Summary Type": [], "Score": []}
finals=[] #list of dictionaries for each video
# Process each summary in the DataFrame
for index, row in df.iterrows():
    data = {"Evaluation Type": [], "Summary Type": [], "Score": []}
    document = row['input_text']
    summary = row['summaries']
    summary_type = f"Summary {index+1}"  # Dynamic summary naming

    # Apply each evaluation metric to the current summary
    for eval_type, (criteria, steps) in evaluation_metrics.items():
        result = get_geval_score(criteria, steps, document, summary, eval_type)
        score_num = float(result.strip())  # Convert to float to handle scores like '4.5'
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summary_type)
        data["Score"].append(score_num)
    finals.append(data)

In [ ]:
finals

[{'Evaluation Type': ['Relevance', 'Coherence', 'Consistency', 'Fluency'],
  'Summary Type': ['Summary 1', 'Summary 1', 'Summary 1', 'Summary 1'],
  'Score': [2.0, 2.0, 2.0, 1.0]},
 {'Evaluation Type': ['Relevance', 'Coherence', 'Consistency', 'Fluency'],
  'Summary Type': ['Summary 2', 'Summary 2', 'Summary 2', 'Summary 2'],
  'Score': [3.5, 2.0, 3.5, 1.0]},
 {'Evaluation Type': ['Relevance', 'Coherence', 'Consistency', 'Fluency'],
  'Summary Type': ['Summary 3', 'Summary 3', 'Summary 3', 'Summary 3'],
  'Score': [3.0, 2.0, 4.0, 1.0]},
 {'Evaluation Type': ['Relevance', 'Coherence', 'Consistency', 'Fluency'],
  'Summary Type': ['Summary 4', 'Summary 4', 'Summary 4', 'Summary 4'],
  'Score': [3.0, 2.0, 3.5, 1.0]},
 {'Evaluation Type': ['Relevance', 'Coherence', 'Consistency', 'Fluency'],
  'Summary Type': ['Summary 5', 'Summary 5', 'Summary 5', 'Summary 5'],
  'Score': [3.0, 2.0, 4.5, 1.0]},
 {'Evaluation Type': ['Relevance', 'Coherence', 'Consistency', 'Fluency'],
  'Summary Type': ['

In [ ]:
# Flatten the list of dictionaries into a single DataFrame
def highlight_max(s):
    is_max = s == s.max()
    return [
        "background-color: lightgreen" if v else "background-color: white"
        for v in is_max
    ]

all_data = []
for entry in finals:
    for eval_type, summ_type, score in zip(entry['Evaluation Type'], entry['Summary Type'], entry['Score']):
        all_data.append({'Evaluation Type': eval_type, 'Summary Type': summ_type, 'Score': score})

df = pd.DataFrame(all_data)

# Pivot the DataFrame
pivot_df = df.pivot(index='Evaluation Type', columns='Summary Type', values='Score')


pivot_df = pivot_df.round(0).astype(int)

# Display the rounded DataFrame
print(pivot_df)



Summary Type     Summary 1  Summary 10  Summary 2  Summary 3  Summary 4  \
Evaluation Type                                                           
Coherence                2           2          2          2          2   
Consistency              2           5          4          4          4   
Fluency                  1           1          1          1          1   
Relevance                2           4          4          3          3   

Summary Type     Summary 5  Summary 6  Summary 7  Summary 8  Summary 9  
Evaluation Type                                                         
Coherence                2          2          2          2          4  
Consistency              4          4          4          2          4  
Fluency                  1          1          1          1          2  
Relevance                3          2          2          2          4  


In [ ]:
pivot_df

Summary Type,Summary 1,Summary 10,Summary 2,Summary 3,Summary 4,Summary 5,Summary 6,Summary 7,Summary 8,Summary 9
Evaluation Type,,,,,,,,,,
Coherence,2,2,2,2,2,2,2,2,2,4
Consistency,2,5,4,4,4,4,4,4,2,4
Fluency,1,1,1,1,1,1,1,1,1,2
Relevance,2,4,4,3,3,3,2,2,2,4


In [ ]:
pivot_df.to_csv('/content/drive/My Drive/LLM_Project/YT/G_Eval_metrics_youtube_videos_bart_model.csv')

In [ ]:
df=pd.read_csv('/content/drive/My Drive/LLM_Project/YT/G_Eval_metrics_youtube_videos_bart_model.csv')

In [ ]:
df

,Evaluation Type,Summary 1,Summary 10,Summary 2,Summary 3,Summary 4,Summary 5,Summary 6,Summary 7,Summary 8,Summary 9
0,Coherence,2,2,2,2,2,2,2,2,2,4
1,Consistency,2,5,4,4,4,4,4,4,2,4
2,Fluency,1,1,1,1,1,1,1,1,1,2
3,Relevance,2,4,4,3,3,3,2,2,2,4


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#transpose the dataframe to have the 'Evaluation Type' as columns
df_transposed = df.set_index('Evaluation Type').transpose()
df_transposed

Evaluation Type,Coherence,Consistency,Fluency,Relevance
Summary 1,2,2,1,2
Summary 10,2,5,1,4
Summary 2,2,4,1,4
Summary 3,2,4,1,3
Summary 4,2,4,1,3
Summary 5,2,4,1,3
Summary 6,2,4,1,2
Summary 7,2,4,1,2
Summary 8,2,2,1,2
Summary 9,4,4,2,4


In [ ]:
df_normalized = scaler.fit_transform(df_transposed)
df_normalized = pd.DataFrame(df_normalized, columns=df_transposed.columns)
df_normalized

Evaluation Type,Coherence,Consistency,Fluency,Relevance
0,0.0,0.000000,0.0,0.0
1,0.0,1.000000,0.0,1.0
2,0.0,0.666667,0.0,1.0
3,0.0,0.666667,0.0,0.5
4,0.0,0.666667,0.0,0.5
5,0.0,0.666667,0.0,0.5
6,0.0,0.666667,0.0,0.0
7,0.0,0.666667,0.0,0.0
8,0.0,0.000000,0.0,0.0
9,1.0,0.666667,1.0,1.0


In [ ]:
# Calculate the mean normalized score for each evaluation type
mean_normalized_scores = round(df_normalized.mean(),3)
print(mean_normalized_scores)

Evaluation Type
Coherence      0.100
Consistency    0.567
Fluency        0.100
Relevance      0.450
dtype: float64


In [ ]:
mean_normalized_scores.name = 'Score'

# Now, convert the Series to a DataFrame
df = mean_normalized_scores.to_frame()
df

,Score
Evaluation Type,
Coherence,0.100
Consistency,0.567
Fluency,0.100
Relevance,0.450


In [ ]:
df.to_csv('/content/drive/My Drive/LLM_Project/YT/normalised_metrics_bart.csv')